# Defensive Distillation

The author's of [Distillation as a Defense to Adversarial
Perturbations against Deep Neural Networks](https://arxiv.org/pdf/1511.04508#page=16&zoom=100,416,109) gives a discription of four key ideas behind distilling image classifiers as a defense against adversarial examples. 

1. Start with hard labels (they describe this a series of one-hot vectors, but that is not necessarily how they would be stored in memory).
2. Train the initial model using a traditional procedure, but let the final layer have a softmax with a temperature greater than one.
3. Create a new training set using the outputs of this initial model. That is, instead of starting with hard labels like the previous model, we start with soft labels outputed by the initial model.
4. Train a new model from scratch using the same architecture but with the soft labels (and with the same temperature as before).


In this notebook, you will implement the final 2 steps and evaluate the results.

In [1]:
from huggingface_hub import hf_hub_download

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Flatten, Linear, ReLU
import xlab

device = xlab.utils.get_best_device()

### Step 1 and 2: Train an image classifer on hard labels

We have already completed this step for you. We trained a simple MLP on the MNIST dataset on for two epochs and achieved a 94.90% accuracy on the test set. Importantly, we use a softmax with temperature ($T=20$) as it is described on our [explainer page](https://xlabaisecurity.com/adversarial/defensive-distillation/).


If interested you can see the output of our training run [here](https://github.com/zroe1/xlab-ai-security/blob/main/models/defensive_distillation/training_output.txt) and the complete code [here](https://github.com/zroe1/xlab-ai-security/tree/main/models/defensive_distillation). You will train your own version of this model for step 5 of this notebook.



In [18]:
# skeleton of the model we trained
class FeedforwardMNIST(nn.Module):
    """Simple 4-layer MLP for MNIST classification"""
    def __init__(self, num_classes=10):
        super(FeedforwardMNIST, self).__init__()
        
        input_size = 28 * 28
        self.fc1 = Linear(input_size, 256)
        self.fc2 = Linear(256, 64)
        self.fc3 = Linear(64, num_classes)
        
        self.flatten = Flatten()
        self.relu = ReLU()
        
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model_path = hf_hub_download(repo_id="uchicago-xlab-ai-security/base-mnist-model", filename="mnist_mlp_temp_30.pth")
model = torch.load(model_path, map_location=device, weights_only=False)

mnist_mlp_temp_30.pth:   0%|          | 0.00/877k [00:00<?, ?B/s]

In [19]:
print(model)

FeedforwardMNIST(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
)


### Benchmark on PGD

In [20]:
num_test_imgs = 30
imgs, ys = xlab.utils.load_mnist_test_samples(num_test_imgs)
loss_fn = torch.nn.CrossEntropyLoss()

num_success = 0

for img, y in zip(imgs, ys):
    adv_x = xlab.utils.PGD(model, loss_fn, img, y, epsilon=32/255, alpha=2/255, num_iters=100)
    adv_y = torch.argmax(model(adv_x))
    
    if adv_y.item() != y:
        num_success += 1

print(f"{(num_success / num_test_imgs) * 100:.4}% of attacks succeded")

30.0% of attacks succeded


## Step 3: Create new training set

We will be training our distilled model on the labels of the pretrained model you have loaded above. 


The model you loaded however, gives logits, not a temperature-smoothed softmax, so to get the proper labels, you will first have to implement the function below which returns softmax with temperature.

In [21]:
train_loader = xlab.utils.get_mnist_train_loader(batch_size=128, shuffle=True)

In [22]:
def softmax_with_temp(inputs, T):
    out = inputs / T
    return F.softmax(out, dim=1)

In [23]:
def get_batch_labels(batch, T):
    outs = model(batch)
    outs = softmax_with_temp(outs, T)
    return outs

In [24]:
from torch.utils.data import DataLoader, TensorDataset

In [25]:
imgs = []
soft_labels= []

with torch.no_grad():
    for x_batch, _ in train_loader:
        x_batch = x_batch.to(device)
        soft_labels_batch = get_batch_labels(x_batch, 30)
    
        imgs.append(x_batch.cpu())
        soft_labels.append(soft_labels_batch.cpu())

all_images = torch.cat(imgs, dim=0)
all_soft_labels = torch.cat(soft_labels, dim=0)
soft_label_dataset = TensorDataset(all_images, all_soft_labels)

batch_size = 128
soft_label_loader = DataLoader(
    soft_label_dataset, 
    batch_size=batch_size, 
    shuffle=True,
)

In [26]:
soft_label_loader

The first step in contructing this new dataset is to implement `get_batch_labels` by calling the pretrained model with temperature T. 

## Step 4: Train distilled model

The optimization problem from the original paper was formalized by the authors using the following equation:

$$
\arg\min_{\theta_F} -\frac{1}{|\mathcal{X}|} \sum_{X \in \mathcal{X}} \sum_{i \in 0..N} F_i(X) \log F_i^d(X)
$$

The loss for a single example is simply cross entropy loss with soft labels:

$$
\mathcal{L}(X) = -\sum_{i \in 0..N} F_i(X) \log F_i^d(X)
$$

In [27]:
# skeleton of the model we trained
distilled =  FeedforwardMNIST().to(device)

In [28]:
def cross_entropy_loss_soft(soft_labels, probs):
    assert soft_labels.shape == probs.shape

    log_probs = torch.log(probs)
    return torch.sum(-1 * log_probs *  soft_labels) / (batch_size * 10)

In [29]:
def train(model, epochs, train_loader, T):
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

    for epoch in range(epochs):
        for i, (img, soft_label) in enumerate(train_loader):
            optimizer.zero_grad()
            
            img, soft_label = img.to(device), soft_label.to(device)
            logits = model(img)
    
            out = softmax_with_temp(logits, T)
            batch_loss = cross_entropy_loss_soft(soft_label, out)
    
            if i % 50==0:
                print(f"Epoch #{epoch + 1}: batch loss = {batch_loss.item():.4f}")
    
            batch_loss.backward()
            optimizer.step()

In [30]:
train(distilled, 10, soft_label_loader, 30)

Epoch #1: batch loss = 0.2303
Epoch #1: batch loss = 0.0559
Epoch #1: batch loss = 0.0363
Epoch #1: batch loss = 0.0342
Epoch #1: batch loss = 0.0348
Epoch #1: batch loss = 0.0305
Epoch #1: batch loss = 0.0310
Epoch #1: batch loss = 0.0264
Epoch #1: batch loss = 0.0254
Epoch #1: batch loss = 0.0202
Epoch #2: batch loss = 0.0268
Epoch #2: batch loss = 0.0308
Epoch #2: batch loss = 0.0190
Epoch #2: batch loss = 0.0227
Epoch #2: batch loss = 0.0224
Epoch #2: batch loss = 0.0234
Epoch #2: batch loss = 0.0248
Epoch #2: batch loss = 0.0312
Epoch #2: batch loss = 0.0235
Epoch #2: batch loss = 0.0214
Epoch #3: batch loss = 0.0273
Epoch #3: batch loss = 0.0237
Epoch #3: batch loss = 0.0265
Epoch #3: batch loss = 0.0272
Epoch #3: batch loss = 0.0234
Epoch #3: batch loss = 0.0262
Epoch #3: batch loss = 0.0246
Epoch #3: batch loss = 0.0247
Epoch #3: batch loss = 0.0310
Epoch #3: batch loss = 0.0261
Epoch #4: batch loss = 0.0261
Epoch #4: batch loss = 0.0235
Epoch #4: batch loss = 0.0189
Epoch #4: 

In [31]:
clean_acc = xlab.utils.evaluate_mnist_accuracy(distilled)
print(f"Clean accuracy of distilled model: {clean_acc*100:.2f}%")

Clean accuracy of distilled model: 93.90%


In [32]:
num_test_imgs = 30
imgs, ys = xlab.utils.load_mnist_test_samples(num_test_imgs)
loss_fn = torch.nn.CrossEntropyLoss()

num_success = 0

for img, y in zip(imgs, ys):
    adv_x = xlab.utils.PGD(distilled, loss_fn, img, y, epsilon=24/255, alpha=2/255, num_iters=100)
    adv_y = torch.argmax(distilled(adv_x))
    
    if adv_y.item() != y:
        num_success += 1

print(f"{(num_success / num_test_imgs) * 100:.4}% of attacks succeded")

23.33% of attacks succeded


In [33]:
model_path = hf_hub_download(repo_id="uchicago-xlab-ai-security/base-mnist-model", filename="mnist_mlp_temp_1.pth")
standard = torch.load(model_path, map_location=device, weights_only=False)

mnist_mlp_temp_1.pth:   0%|          | 0.00/877k [00:00<?, ?B/s]

In [34]:
clean_acc = xlab.utils.evaluate_mnist_accuracy(standard)
print(f"Clean accuracy of standard model: {clean_acc*100:.2f}%")

num_test_imgs = 30
imgs, ys = xlab.utils.load_mnist_test_samples(num_test_imgs)
loss_fn = torch.nn.CrossEntropyLoss()

num_success = 0

for img, y in zip(imgs, ys):
    adv_x = xlab.utils.PGD(standard, loss_fn, img, y, epsilon=24/255, alpha=2/255, num_iters=100)
    adv_y = torch.argmax(standard(adv_x))
    
    if adv_y.item() != y:
        num_success += 1

print(f"{(num_success / num_test_imgs) * 100:.4}% of attacks succeded")

Clean accuracy of standard model: 96.62%
100.0% of attacks succeded
